### Step 40: Regenerate SWC PREC Datasets

In [3]:
import arcpy,os; 

fgdb    = os.getcwd() + os.sep + 'source.gdb';
rezfgdb = os.getcwd() + os.sep + 'results.gdb';
wrkfgdb = os.getcwd() + os.sep + 'working.gdb';

if not arcpy.Exists(rezfgdb):
   arcpy.CreateFileGDB_management(
       os.path.dirname(rezfgdb)
      ,os.path.basename(rezfgdb)
   );

wrkstat   = wrkfgdb + os.sep + 'D4EMLite_PREC_Details_WRK';
stat_cnt  = arcpy.GetCount_management(wrkstat)[0];

grid      = fgdb + os.sep + 'CRWU_CREAT_Grid_Projections';
grid_cnt  = arcpy.GetCount_management(grid)[0];

hist      = fgdb + os.sep + 'CRWU_CREAT_Historic_Climate_Stations';
hist_cnt  = arcpy.GetCount_management(hist)[0];

rfcov     = rezfgdb + os.sep + 'precipmap';
rfcov_cnt = arcpy.GetCount_management(rfcov)[0];

print("Station Count          : " + str(stat_cnt));
print("Grid Count             : " + str(grid_cnt));
print("Historic Station Count : " + str(hist_cnt));
print("Precipitation Map Count: " + str(rfcov_cnt));


Station Count          : 5144
Grid Count             : 24743
Historic Station Count : 11165
Precipitation Map Count: 19593


### Calculate nearest historical station

In [4]:
wrkstat2 = wrkfgdb + os.sep + 'D4EMLite_PREC_Details_WRK2';

if arcpy.Exists(wrkstat2):
    arcpy.Delete_management(wrkstat2);
   
print("  copying to new work stations");
arcpy.conversion.FeatureClassToFeatureClass(
     in_features = wrkstat
    ,out_path    = wrkfgdb
    ,out_name    = 'D4EMLite_PREC_Details_WRK2'
);

print("  calculating the nearest historic station");
arcpy.analysis.Near(
     in_features   = wrkstat2
    ,near_features = hist
    ,search_radius = '150 kilometers'
    ,location      = 'NO_LOCATION'
    ,method        = 'GEODESIC'
);

print("  updating fieldnames");
arcpy.management.AlterField(
     in_table        = wrkstat2
    ,field           = 'NEAR_FID'
    ,new_field_name  = 'NEAR_FID_HIST'
    ,new_field_alias = 'NEAR_FID_HIST'
);

arcpy.management.AlterField(
     in_table        = wrkstat2
    ,field           = 'NEAR_DIST'
    ,new_field_name  = 'NEAR_DIST_HIST'
    ,new_field_alias = 'NEAR_DIST_HIST'
);

print("  adding indexes");
arcpy.management.AddIndex(
     in_table        = wrkstat2
    ,fields          = 'NEAR_FID_HIST'
    ,index_name      = 'NEAR_FID_HIST_IDX'
);

print("  joining in historic storms information");
arcpy.management.JoinField(
     in_data         = wrkstat2
    ,in_field        = 'NEAR_FID_HIST'
    ,join_table      = hist
    ,join_field      = 'OBJECTID'
    ,fields          = [
         'IntensePrecip_Historic_5Year_24'
        ,'IntensePrecip_Historic_10Year_2'
        ,'IntensePrecip_Historic_15Year_2'
        ,'IntensePrecip_Historic_30Year_2'
        ,'IntensePrecip_Historic_50Year_2'
        ,'IntensePrecip_Historic_100Year_'
    ]
);

print("DONE");


  copying to new work stations
  calculating the nearerest historic station
  updating fieldnames
  adding indexes
  joining in historic storms information
DONE


## Determine CREAT3 grid for stations

In [5]:
wrkstat3 = wrkfgdb + os.sep + 'D4EMLite_PREC_Details_WRK3';

if arcpy.Exists(wrkstat3):
    arcpy.Delete_management(wrkstat3);

print("  building spatial join with CREAT3 grid");
arcpy.analysis.SpatialJoin(
     target_features   = wrkstat2
    ,join_features     = grid
    ,out_feature_class = wrkstat3
    ,join_operation    = 'JOIN_ONE_TO_ONE'
    ,join_type         = 'KEEP_ALL'
    ,match_option      = 'INTERSECT'
);

print("DONE");


  building spatial join with CREAT3 grid
DONE


### Determine Precipitation RF value for stations

In [6]:
wrkstat4 = wrkfgdb + os.sep + 'D4EMLite_PREC_Details_WRK4';

if arcpy.Exists(wrkstat4):
    arcpy.Delete_management(wrkstat4);

print("  building spatial join with precipitation coverage");
arcpy.analysis.SpatialJoin(
     target_features   = wrkstat3
    ,join_features     = rfcov
    ,out_feature_class = wrkstat4
    ,join_operation    = 'JOIN_ONE_TO_ONE'
    ,join_type         = 'KEEP_ALL'
    ,match_option      = 'INTERSECT'
);

print("DONE");


  building spatial join with precipitation coverage
DONE


### Generate tables for PREC extracts 

In [13]:
def createPREC(fgdb,fc):
    
    if arcpy.Exists(fgdb + os.sep + fc):
        arcpy.Delete_management(fgdb + os.sep + fc);
    
    arcpy.CreateFeatureclass_management(
         out_path      = fgdb
        ,out_name      = fc
        ,geometry_type = "POINT"
        ,has_m         = "DISABLED"
        ,has_z         = "DISABLED"
        ,spatial_reference = arcpy.SpatialReference(4269) 
    );
    
    arcpy.management.AddFields(
         in_table = fgdb + os.sep + fc
        ,field_description = [
             ['StationId','TEXT'  ,'StationId',14]
            ,['Jan'      ,'DOUBLE','Jan']
            ,['Feb'      ,'DOUBLE','Feb']
            ,['Mar'      ,'DOUBLE','Mar']
            ,['Apr'      ,'DOUBLE','Apr']
            ,['May'      ,'DOUBLE','May']
            ,['Jun'      ,'DOUBLE','Jun']
            ,['Jul'      ,'DOUBLE','Jul']
            ,['Aug'      ,'DOUBLE','Aug']
            ,['Sep'      ,'DOUBLE','Sep']
            ,['Oct'      ,'DOUBLE','Oct']
            ,['Nov'      ,'DOUBLE','Nov']
            ,['Dec'      ,'DOUBLE','Dec']
            ,['Ann'      ,'DOUBLE','Ann']
         ]
    );
    
def createGEVdepth(fgdb,fc):
    
    if arcpy.Exists(fgdb + os.sep + fc):
        arcpy.Delete_management(fgdb + os.sep + fc);
    
    arcpy.CreateFeatureclass_management(
         out_path      = fgdb
        ,out_name      = fc
        ,geometry_type = "POINT"
        ,has_m         = "DISABLED"
        ,has_z         = "DISABLED"
        ,spatial_reference = arcpy.SpatialReference(4269) 
    );
    
    arcpy.management.AddFields(
         in_table = fgdb + os.sep + fc
        ,field_description = [
             ['StationId'   ,'TEXT'  ,'StationId',14]
            ,['z5yrann_mm'  ,'DOUBLE','5yrann_mm']
            ,['z10yrann_mm' ,'DOUBLE','10yrann_mm']
            ,['z15yrann_mm' ,'DOUBLE','15yrann_mm']
            ,['z30yrann_mm' ,'DOUBLE','30yrann_mm']
            ,['z50yrann_mm' ,'DOUBLE','50yrann_mm']
            ,['z100yrann_mm','DOUBLE','100yrann_mm']
         ]
    );
    
createPREC(rezfgdb,'PREC2035Hot');
createPREC(rezfgdb,'PREC2035Med');
createPREC(rezfgdb,'PREC2035Wet');
createPREC(rezfgdb,'PREC2060Hot');
createPREC(rezfgdb,'PREC2060Med');
createPREC(rezfgdb,'PREC2060Wet');

createGEVdepth(rezfgdb,'GEVdepth2035Stormy');
createGEVdepth(rezfgdb,'GEVdepth2035LessStormy');
createGEVdepth(rezfgdb,'GEVdepth2060Stormy');
createGEVdepth(rezfgdb,'GEVdepth2060LessStormy');
createGEVdepth(rezfgdb,'GEVdepthHistoric');

if arcpy.Exists(rezfgdb + os.sep + 'PREC_SCS_Types'):
    arcpy.Delete_management(rezfgdb + os.sep + 'PREC_SCS_Types');
        
arcpy.CreateFeatureclass_management(
     out_path      = rezfgdb
    ,out_name      = 'PREC_SCS_Types'
    ,geometry_type = "POINT"
    ,has_m         = "DISABLED"
    ,has_z         = "DISABLED"
    ,spatial_reference = arcpy.SpatialReference(4269) 
);

arcpy.management.AddFields(
     in_table = rezfgdb + os.sep + 'PREC_SCS_Types'
    ,field_description = [
         ['StationId'                 ,'TEXT','StationId',14]
        ,['Rainfall_Distribution_Type','TEXT','Rainfall_Distribution_Type',255]
     ]
);

print("DONE");


DONE


In [15]:
fldin = [
     'StationId'
    
    ,'PRECIP_2035_HOTDRY_JAN_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_FEB_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_MAR_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_APR_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_MAY_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_JUN_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_JUL_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_AUG_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_SEP_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_OCT_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_NOV_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_DEC_CHANGEPC'
    ,'PRECIP_2035_HOTDRY_ANNUAL_CHANG'
    
    ,'PRECIP_2035_CENTRAL_JAN_CHANGEP'
    ,'PRECIP_2035_CENTRAL_FEB_CHANGEP'
    ,'PRECIP_2035_CENTRAL_MAR_CHANGEP'
    ,'PRECIP_2035_CENTRAL_APR_CHANGEP'
    ,'PRECIP_2035_CENTRAL_MAY_CHANGEP'
    ,'PRECIP_2035_CENTRAL_JUN_CHANGEP'
    ,'PRECIP_2035_CENTRAL_JUL_CHANGEP'
    ,'PRECIP_2035_CENTRAL_AUG_CHANGEP'
    ,'PRECIP_2035_CENTRAL_SEP_CHANGEP'
    ,'PRECIP_2035_CENTRAL_OCT_CHANGEP'
    ,'PRECIP_2035_CENTRAL_NOV_CHANGEP'
    ,'PRECIP_2035_CENTRAL_DEC_CHANGEP'
    ,'PRECIP_2035_CENTRAL_ANNUAL_CHAN'
    
    ,'PRECIP_2035_WETWARM_JAN_CHANGEP'
    ,'PRECIP_2035_WETWARM_FEB_CHANGEP'
    ,'PRECIP_2035_WETWARM_MAR_CHANGEP'
    ,'PRECIP_2035_WETWARM_APR_CHANGEP'
    ,'PRECIP_2035_WETWARM_MAY_CHANGEP'
    ,'PRECIP_2035_WETWARM_JUN_CHANGEP'
    ,'PRECIP_2035_WETWARM_JUL_CHANGEP'
    ,'PRECIP_2035_WETWARM_AUG_CHANGEP'
    ,'PRECIP_2035_WETWARM_SEP_CHANGEP'
    ,'PRECIP_2035_WETWARM_OCT_CHANGEP'
    ,'PRECIP_2035_WETWARM_NOV_CHANGEP'
    ,'PRECIP_2035_WETWARM_DEC_CHANGEP'
    ,'PRECIP_2035_WETWARM_ANNUAL_CHAN'
    
    ,'PRECIP_2060_HOTDRY_JAN_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_FEB_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_MAR_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_APR_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_MAY_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_JUN_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_JUL_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_AUG_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_SEP_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_OCT_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_NOV_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_DEC_CHANGEPC'
    ,'PRECIP_2060_HOTDRY_ANNUAL_CHANG'
    
    ,'PRECIP_2060_CENTRAL_JAN_CHANGEP'
    ,'PRECIP_2060_CENTRAL_FEB_CHANGEP'
    ,'PRECIP_2060_CENTRAL_MAR_CHANGEP'
    ,'PRECIP_2060_CENTRAL_APR_CHANGEP'
    ,'PRECIP_2060_CENTRAL_MAY_CHANGEP'
    ,'PRECIP_2060_CENTRAL_JUN_CHANGEP'
    ,'PRECIP_2060_CENTRAL_JUL_CHANGEP'
    ,'PRECIP_2060_CENTRAL_AUG_CHANGEP'
    ,'PRECIP_2060_CENTRAL_SEP_CHANGEP'
    ,'PRECIP_2060_CENTRAL_OCT_CHANGEP'
    ,'PRECIP_2060_CENTRAL_NOV_CHANGEP'
    ,'PRECIP_2060_CENTRAL_DEC_CHANGEP'
    ,'PRECIP_2060_CENTRAL_ANNUAL_CHAN'
    
    ,'PRECIP_2060_WETWARM_JAN_CHANGEP'
    ,'PRECIP_2060_WETWARM_FEB_CHANGEP'
    ,'PRECIP_2060_WETWARM_MAR_CHANGEP'
    ,'PRECIP_2060_WETWARM_APR_CHANGEP'
    ,'PRECIP_2060_WETWARM_MAY_CHANGEP'
    ,'PRECIP_2060_WETWARM_JUN_CHANGEP'
    ,'PRECIP_2060_WETWARM_JUL_CHANGEP'
    ,'PRECIP_2060_WETWARM_AUG_CHANGEP'
    ,'PRECIP_2060_WETWARM_SEP_CHANGEP'
    ,'PRECIP_2060_WETWARM_OCT_CHANGEP'
    ,'PRECIP_2060_WETWARM_NOV_CHANGEP'
    ,'PRECIP_2060_WETWARM_DEC_CHANGEP'
    ,'PRECIP_2060_WETWARM_ANNUAL_CHAN'
    
    ,'STORM5YR_2035_STORMY_ANNUAL_CHA'
    ,'STORM10YR_2035_STORMY_ANNUAL_CH'
    ,'STORM15YR_2035_STORMY_ANNUAL_CH'
    ,'STORM30YR_2035_STORMY_ANNUAL_CH'
    ,'STORM50YR_2035_STORMY_ANNUAL_CH'
    ,'STORM100YR_2035_STORMY_ANNUAL_C'
    
    ,'STORM5YR_2060_STORMY_ANNUAL_CHA'
    ,'STORM10YR_2060_STORMY_ANNUAL_CH'
    ,'STORM15YR_2060_STORMY_ANNUAL_CH'
    ,'STORM30YR_2060_STORMY_ANNUAL_CH'
    ,'STORM50YR_2060_STORMY_ANNUAL_CH'
    ,'STORM100YR_2060_STORMY_ANNUAL_C'
    
    ,'STORM5YR_2035_LESSSTORMY_ANNUAL'
    ,'STORM10YR_2035_LESSSTORMY_ANNUA'
    ,'STORM15YR_2035_LESSSTORMY_ANNUA'
    ,'STORM30YR_2035_LESSSTORMY_ANNUA'
    ,'STORM50YR_2035_LESSSTORMY_ANNUA'
    ,'STORM100YR_2035_LESSSTORMY_ANNU'
    
    ,'STORM5YR_2060_LESSSTORMY_ANNUAL'
    ,'STORM10YR_2060_LESSSTORMY_ANNUA'
    ,'STORM15YR_2060_LESSSTORMY_ANNUA'
    ,'STORM30YR_2060_LESSSTORMY_ANNUA'
    ,'STORM50YR_2060_LESSSTORMY_ANNUA'
    ,'STORM100YR_2060_LESSSTORMY_ANNU'
    
    ,'IntensePrecip_Historic_5Year_24'
    ,'IntensePrecip_Historic_10Year_2'
    ,'IntensePrecip_Historic_15Year_2'
    ,'IntensePrecip_Historic_30Year_2'
    ,'IntensePrecip_Historic_50Year_2'
    ,'IntensePrecip_Historic_100Year_'
    
    ,'rf_value'
];

fldprec = [
     'StationId'
    ,'Jan'
    ,'Feb'
    ,'Mar'
    ,'Apr'
    ,'May'
    ,'Jun'
    ,'Jul'
    ,'Aug'
    ,'Sep'
    ,'Oct'
    ,'Nov'
    ,'Dec'
    ,'Ann'
];

fldgev = [
     'StationId'
    ,'z5yrann_mm'
    ,'z10yrann_mm'
    ,'z15yrann_mm'
    ,'z30yrann_mm'
    ,'z50yrann_mm'
    ,'z100yrann_mm'
];

fldscs = [
     'StationId'
    ,'Rainfall_Distribution_Type'
]

print("  opening an editor session to support multiple cursors")
with arcpy.da.Editor(rezfgdb) as edit:
      
    curs_PREC2035Hot = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'PREC2035Hot'   
        ,field_names = fldprec
    );
    
    curs_PREC2035Med = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'PREC2035Med'   
        ,field_names = fldprec
    );
        
    curs_PREC2035Wet = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'PREC2035Wet'   
        ,field_names = fldprec
    );
    
    curs_PREC2060Hot = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'PREC2060Hot'   
        ,field_names = fldprec
    );
    
    curs_PREC2060Med = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'PREC2060Med'   
        ,field_names = fldprec
    );
        
    curs_PREC2060Wet = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'PREC2060Wet'   
        ,field_names = fldprec
    );
    
    curs_GEVdepth2035Stormy = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'GEVdepth2035Stormy'   
        ,field_names = fldgev
    );
        
    curs_GEVdepth2035LessStormy = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'GEVdepth2035LessStormy'   
        ,field_names = fldgev
    );
    
    curs_GEVdepth2060Stormy = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'GEVdepth2060Stormy'   
        ,field_names = fldgev
    );
        
    curs_GEVdepth2060LessStormy = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'GEVdepth2060LessStormy'   
        ,field_names = fldgev
    );
    
    curs_GEVdepthHistoric = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'GEVdepthHistoric'   
        ,field_names = fldgev
    );
    
    curs_PREC_SCS_Types = arcpy.da.InsertCursor(
         in_table    = rezfgdb + os.sep + 'PREC_SCS_Types'   
        ,field_names = fldscs
    );

    with arcpy.da.SearchCursor(wrkstat4,fldin) as incur:
    
        for row in incur:
            
            curs_PREC2035Hot.insertRow((
                 row[0]
                ,row[1]
                ,row[2]
                ,row[3]
                ,row[4]
                ,row[5]
                ,row[6]
                ,row[7]
                ,row[8]
                ,row[9]
                ,row[10]
                ,row[11]
                ,row[12]
                ,row[13]
            ));
            
            curs_PREC2035Med.insertRow((
                 row[0]
                ,row[14]
                ,row[15]
                ,row[16]
                ,row[17]
                ,row[18]
                ,row[19]
                ,row[20]
                ,row[21]
                ,row[22]
                ,row[23]
                ,row[24]
                ,row[25]
                ,row[26]
            ));
            
            curs_PREC2035Wet.insertRow((
                 row[0]
                ,row[27]
                ,row[28]
                ,row[29]
                ,row[30]
                ,row[31]
                ,row[32]
                ,row[33]
                ,row[34]
                ,row[35]
                ,row[36]
                ,row[37]
                ,row[38]
                ,row[39]
            ));
            
            curs_PREC2060Hot.insertRow((
                 row[0]
                ,row[40]
                ,row[41]
                ,row[42]
                ,row[43]
                ,row[44]
                ,row[45]
                ,row[46]
                ,row[47]
                ,row[48]
                ,row[49]
                ,row[50]
                ,row[51]
                ,row[52]
            ));
            
            curs_PREC2060Med.insertRow((
                 row[0]
                ,row[53]
                ,row[54]
                ,row[55]
                ,row[56]
                ,row[57]
                ,row[58]
                ,row[59]
                ,row[60]
                ,row[61]
                ,row[62]
                ,row[63]
                ,row[64]
                ,row[65]
            ));
            
            curs_PREC2060Wet.insertRow((
                 row[0]
                ,row[66]
                ,row[67]
                ,row[68]
                ,row[69]
                ,row[70]
                ,row[71]
                ,row[72]
                ,row[73]
                ,row[74]
                ,row[75]
                ,row[76]
                ,row[77]
                ,row[78]
            ));
            
            curs_GEVdepth2035Stormy.insertRow((
                 row[0]
                ,row[79]
                ,row[80]
                ,row[81]
                ,row[82]
                ,row[83]
                ,row[84]
            ));
            
            curs_GEVdepth2035LessStormy.insertRow((
                 row[0]
                ,row[85]
                ,row[86]
                ,row[87]
                ,row[88]
                ,row[89]
                ,row[90]
            ));
            
            curs_GEVdepth2060Stormy.insertRow((
                 row[0]
                ,row[91]
                ,row[92]
                ,row[93]
                ,row[94]
                ,row[95]
                ,row[96]
            ));
            
            curs_GEVdepth2060LessStormy.insertRow((
                 row[0]
                ,row[97]
                ,row[98]
                ,row[99]
                ,row[100]
                ,row[101]
                ,row[102]
            ));
            
            curs_GEVdepthHistoric.insertRow((
                 row[0]
                ,row[103]
                ,row[104]
                ,row[105]
                ,row[106]
                ,row[107]
                ,row[108]
            ));
            
            curs_PREC_SCS_Types.insertRow((
                 row[0]
                ,row[109]
            ));
 
print("DONE");


  opening an editor session to support multiple cursors
DONE


In [16]:
stat = rezfgdb + os.sep + 'D4EMLite_PREC_Details';

if arcpy.Exists(stat):
    arcpy.Delete_management(stat);
   
print("  copying to final workstations");
arcpy.conversion.FeatureClassToFeatureClass(
     in_features = wrkstat3
    ,out_path    = rezfgdb
    ,out_name    = 'D4EMLite_PREC_Details'
);

print("  cleaning up the fieldset");
arcpy.management.DeleteField(
     in_table   = stat
    ,drop_field = [
         'Join_Count'
        ,'TARGET_FID'
        ,'NEAR_FID_HIST'
        ,'NEAR_DIST_HIST'
        ,'IntensePrecip_Historic_5Year_24'
        ,'IntensePrecip_Historic_10Year_2'
        ,'IntensePrecip_Historic_15Year_2'
        ,'IntensePrecip_Historic_30Year_2'
        ,'IntensePrecip_Historic_50Year_2'
        ,'IntensePrecip_Historic_100Year_'
        ,'CREAT_ID'
        ,'GRIDCODE'
        ,'HurrGridID'
        ,'grid_over_land'
        ,'Longitude_Centroid'
        ,'Latitude_Centroid'
        ,'Longitude_Min'
        ,'Latitude_Min'
        ,'Longitude_Max'
        ,'Latitude_Max'
        ,'PRECIP_2035_HOTDRY_JAN_CHANGEPC'
        ,'PRECIP_2035_HOTDRY_FEB_CHANGEPC'
        ,'PRECIP_2035_HOTDRY_MAR_CHANGEPC'
        ,'PRECIP_2035_HOTDRY_APR_CHANGEPC'
        ,'PRECIP_2035_HOTDRY_MAY_CHANGEPC'
        ,'PRECIP_2035_HOTDRY_JUN_CHANGEPC'
        ,'PRECIP_2035_HOTDRY_JUL_CHANGEPC'
        ,'PRECIP_2035_HOTDRY_AUG_CHANGEPC'
        ,'PRECIP_2035_HOTDRY_SEP_CHANGEPC'
        ,'PRECIP_2035_HOTDRY_OCT_CHANGEPC'
        ,'PRECIP_2035_HOTDRY_NOV_CHANGEPC'
        ,'PRECIP_2035_HOTDRY_DEC_CHANGEPC'
        ,'PRECIP_2035_HOTDRY_ANNUAL_CHANG'
        ,'PRECIP_2035_CENTRAL_JAN_CHANGEP'
        ,'PRECIP_2035_CENTRAL_FEB_CHANGEP'
        ,'PRECIP_2035_CENTRAL_MAR_CHANGEP'
        ,'PRECIP_2035_CENTRAL_APR_CHANGEP'
        ,'PRECIP_2035_CENTRAL_MAY_CHANGEP'
        ,'PRECIP_2035_CENTRAL_JUN_CHANGEP'
        ,'PRECIP_2035_CENTRAL_JUL_CHANGEP'
        ,'PRECIP_2035_CENTRAL_AUG_CHANGEP'
        ,'PRECIP_2035_CENTRAL_SEP_CHANGEP'
        ,'PRECIP_2035_CENTRAL_OCT_CHANGEP'
        ,'PRECIP_2035_CENTRAL_NOV_CHANGEP'
        ,'PRECIP_2035_CENTRAL_DEC_CHANGEP'
        ,'PRECIP_2035_CENTRAL_ANNUAL_CHAN'
        ,'PRECIP_2035_WETWARM_JAN_CHANGEP'
        ,'PRECIP_2035_WETWARM_FEB_CHANGEP'
        ,'PRECIP_2035_WETWARM_MAR_CHANGEP'
        ,'PRECIP_2035_WETWARM_APR_CHANGEP'
        ,'PRECIP_2035_WETWARM_MAY_CHANGEP'
        ,'PRECIP_2035_WETWARM_JUN_CHANGEP'
        ,'PRECIP_2035_WETWARM_JUL_CHANGEP'
        ,'PRECIP_2035_WETWARM_AUG_CHANGEP'
        ,'PRECIP_2035_WETWARM_SEP_CHANGEP'
        ,'PRECIP_2035_WETWARM_OCT_CHANGEP'
        ,'PRECIP_2035_WETWARM_NOV_CHANGEP'
        ,'PRECIP_2035_WETWARM_DEC_CHANGEP'
        ,'PRECIP_2035_WETWARM_ANNUAL_CHAN'
        ,'PRECIP_2060_HOTDRY_JAN_CHANGEPC'
        ,'PRECIP_2060_HOTDRY_FEB_CHANGEPC'
        ,'PRECIP_2060_HOTDRY_MAR_CHANGEPC'
        ,'PRECIP_2060_HOTDRY_APR_CHANGEPC'
        ,'PRECIP_2060_HOTDRY_MAY_CHANGEPC'
        ,'PRECIP_2060_HOTDRY_JUN_CHANGEPC'
        ,'PRECIP_2060_HOTDRY_JUL_CHANGEPC'
        ,'PRECIP_2060_HOTDRY_AUG_CHANGEPC'
        ,'PRECIP_2060_HOTDRY_SEP_CHANGEPC'
        ,'PRECIP_2060_HOTDRY_OCT_CHANGEPC'
        ,'PRECIP_2060_HOTDRY_NOV_CHANGEPC'
        ,'PRECIP_2060_HOTDRY_DEC_CHANGEPC'
        ,'PRECIP_2060_HOTDRY_ANNUAL_CHANG'
        ,'PRECIP_2060_CENTRAL_JAN_CHANGEP'
        ,'PRECIP_2060_CENTRAL_FEB_CHANGEP'
        ,'PRECIP_2060_CENTRAL_MAR_CHANGEP'
        ,'PRECIP_2060_CENTRAL_APR_CHANGEP'
        ,'PRECIP_2060_CENTRAL_MAY_CHANGEP'
        ,'PRECIP_2060_CENTRAL_JUN_CHANGEP'
        ,'PRECIP_2060_CENTRAL_JUL_CHANGEP'
        ,'PRECIP_2060_CENTRAL_AUG_CHANGEP'
        ,'PRECIP_2060_CENTRAL_SEP_CHANGEP'
        ,'PRECIP_2060_CENTRAL_OCT_CHANGEP'
        ,'PRECIP_2060_CENTRAL_NOV_CHANGEP'
        ,'PRECIP_2060_CENTRAL_DEC_CHANGEP'
        ,'PRECIP_2060_CENTRAL_ANNUAL_CHAN'
        ,'PRECIP_2060_WETWARM_JAN_CHANGEP'
        ,'PRECIP_2060_WETWARM_FEB_CHANGEP'
        ,'PRECIP_2060_WETWARM_MAR_CHANGEP'
        ,'PRECIP_2060_WETWARM_APR_CHANGEP'
        ,'PRECIP_2060_WETWARM_MAY_CHANGEP'
        ,'PRECIP_2060_WETWARM_JUN_CHANGEP'
        ,'PRECIP_2060_WETWARM_JUL_CHANGEP'
        ,'PRECIP_2060_WETWARM_AUG_CHANGEP'
        ,'PRECIP_2060_WETWARM_SEP_CHANGEP'
        ,'PRECIP_2060_WETWARM_OCT_CHANGEP'
        ,'PRECIP_2060_WETWARM_NOV_CHANGEP'
        ,'PRECIP_2060_WETWARM_DEC_CHANGEP'
        ,'PRECIP_2060_WETWARM_ANNUAL_CHAN'
        ,'STORM5YR_2035_STORMY_ANNUAL_CHA'
        ,'STORM10YR_2035_STORMY_ANNUAL_CH'
        ,'STORM15YR_2035_STORMY_ANNUAL_CH'
        ,'STORM30YR_2035_STORMY_ANNUAL_CH'
        ,'STORM50YR_2035_STORMY_ANNUAL_CH'
        ,'STORM100YR_2035_STORMY_ANNUAL_C'
        ,'STORM5YR_2060_STORMY_ANNUAL_CHA'
        ,'STORM10YR_2060_STORMY_ANNUAL_CH'
        ,'STORM15YR_2060_STORMY_ANNUAL_CH'
        ,'STORM30YR_2060_STORMY_ANNUAL_CH'
        ,'STORM50YR_2060_STORMY_ANNUAL_CH'
        ,'STORM100YR_2060_STORMY_ANNUAL_C'
        ,'STORM5YR_2035_LESSSTORMY_ANNUAL'
        ,'STORM10YR_2035_LESSSTORMY_ANNUA'
        ,'STORM15YR_2035_LESSSTORMY_ANNUA'
        ,'STORM30YR_2035_LESSSTORMY_ANNUA'
        ,'STORM50YR_2035_LESSSTORMY_ANNUA'
        ,'STORM100YR_2035_LESSSTORMY_ANNU'
        ,'STORM5YR_2060_LESSSTORMY_ANNUAL'
        ,'STORM10YR_2060_LESSSTORMY_ANNUA'
        ,'STORM15YR_2060_LESSSTORMY_ANNUA'
        ,'STORM30YR_2060_LESSSTORMY_ANNUA'
        ,'STORM50YR_2060_LESSSTORMY_ANNUA'
        ,'STORM100YR_2060_LESSSTORMY_ANNU'
        ,'state'
        ,'rf_value'
        ,'source'
    ]
);

print("DONE");


  copying to final workstations
  cleaning up the fieldset


In [19]:
D4EMLite_PREC_Details_cnt = arcpy.GetCount_management(rezfgdb + os.sep + 'D4EMLite_PREC_Details')[0];

PREC2035Hot_cnt = arcpy.GetCount_management(rezfgdb + os.sep + 'PREC2035Hot')[0];
PREC2035Med_cnt = arcpy.GetCount_management(rezfgdb + os.sep + 'PREC2035Med')[0];
PREC2035Wet_cnt = arcpy.GetCount_management(rezfgdb + os.sep + 'PREC2035Wet')[0];

PREC2060Hot_cnt = arcpy.GetCount_management(rezfgdb + os.sep + 'PREC2060Hot')[0];
PREC2060Med_cnt = arcpy.GetCount_management(rezfgdb + os.sep + 'PREC2060Med')[0];
PREC2060Wet_cnt = arcpy.GetCount_management(rezfgdb + os.sep + 'PREC2060Wet')[0];

GEVdepth2035Stormy_cnt     = arcpy.GetCount_management(rezfgdb + os.sep + 'GEVdepth2035Stormy')[0];
GEVdepth2035LessStormy_cnt = arcpy.GetCount_management(rezfgdb + os.sep + 'GEVdepth2035LessStormy')[0];

GEVdepth2060Stormy_cnt     = arcpy.GetCount_management(rezfgdb + os.sep + 'GEVdepth2060Stormy')[0];
GEVdepth2060LessStormy_cnt = arcpy.GetCount_management(rezfgdb + os.sep + 'GEVdepth2060LessStormy')[0];
GEVdepthHistoric_cnt       = arcpy.GetCount_management(rezfgdb + os.sep + 'GEVdepthHistoric')[0];

PREC_SCS_Types_cnt         = arcpy.GetCount_management(rezfgdb + os.sep + 'PREC_SCS_Types')[0];

print("D4EMLite_PREC_Details Count : " + str(D4EMLite_PREC_Details_cnt));

print("PREC2035Hot Count           : " + str(PREC2035Hot_cnt));
print("PREC2035Med Count           : " + str(PREC2035Med_cnt));
print("PREC2035Wet Count           : " + str(PREC2035Wet_cnt));

print("PREC2060Hot Count           : " + str(PREC2060Hot_cnt));
print("PREC2060Med Count           : " + str(PREC2060Med_cnt));
print("PREC2060Wet Count           : " + str(PREC2060Wet_cnt));

print("GEVdepth2035Stormy Count    : " + str(GEVdepth2035Stormy_cnt));
print("GEVdepth2035LessStormy Count: " + str(GEVdepth2035LessStormy_cnt));

print("GEVdepth2060Stormy Count    : " + str(GEVdepth2060Stormy_cnt));
print("GEVdepth2060LessStormy Count: " + str(GEVdepth2060LessStormy_cnt));

print("GEVdepthHistoric Count      : " + str(GEVdepthHistoric_cnt));

print("PREC_SCS_Types Count        : " + str(PREC_SCS_Types_cnt));


D4EMLite_PREC_Details Count : 5144
PREC2035Hot Count           : 5144
PREC2035Med Count           : 5144
PREC2035Wet Count           : 5144
PREC2060Hot Count           : 5144
PREC2060Med Count           : 5144
PREC2060Wet Count           : 5144
GEVdepth2035Stormy Count    : 5144
GEVdepth2035LessStormy Count: 5144
GEVdepth2060Stormy Count    : 5144
GEVdepth2060LessStormy Count: 5144
GEVdepthHistoric Count      : 5144
PREC_SCS_Types Count        : 5144
